In [1]:
# Visualization of OFDM with its subcarriers, cos waves only
# finalized version @20180403
# by HP

#from __future__ import print_function
#from ipywidgets import interact, interactive, fixed, interact_manual
%matplotlib inline
import ipywidgets as widgets
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt



### Coloring
# Color from http://www.randalolson.com/2014/06/28/how-to-make-beautiful-data-visualizations-in-python-with-matplotlib/
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
         (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
         (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
         (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
         (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)] 
# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.)  
# from http://www.color-hex.com/color-palette/1294
tableau_blues = [(1,31,75), (3,57,108), (0,91,150), (100,151,177), (179,205,224)]
for i in range(len(tableau_blues)):    
    r, g, b = tableau_blues[i]    
    tableau_blues[i] = (r / 255., g / 255., b / 255.)




def ofdm3d(amp_1=1, phase_1=0.5, 
           amp_2=1, phase_2=0, 
           amp_3=2, phase_3=0, 
           amp_4=1, phase_4=0,
           elev=40, azim=-35,
           t_check=True, f_check=True, 
           ani_step=0, save_check=False
          ):
    
    # To remove part of the margin of the plot-out cube area
    # source: https://stackoverflow.com/questions/16488182/removing-axes-margins-in-3d-plot
    ###patch start###   
    from mpl_toolkits.mplot3d.axis3d import Axis
    if not hasattr(Axis, "_get_coord_info_old"):
        def _get_coord_info_new(self, renderer):
            mins, maxs, centers, deltas, tc, highs = self._get_coord_info_old(renderer)
            mins += deltas / 4
            maxs -= deltas / 4
            return mins, maxs, centers, deltas, tc, highs
        Axis._get_coord_info_old = Axis._get_coord_info  
        Axis._get_coord_info = _get_coord_info_new
    ###patch end###
    
    # This is for solving the latex rendering problem. not very successful
    # https://stackoverflow.com/questions/14324477/bold-font-weight-for-latex-axes-label-in-matplotlib/16019418
    import matplotlib as mpl
    mpl.rc('text', usetex=False)
    #mpl.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]    # latex error, maybe has to do with installation
    
    
    # Config of the figure
    fig = plt.figure(figsize=(12,6), dpi=150)
    ax = fig.gca(projection='3d')



    # Parameters
    amp_list=[amp_1,amp_2,amp_3,amp_4]
    phase_list=[-phase_1,-phase_2,-phase_3,-phase_4]
    points=512
    x_length=12.56    # 12.56 for total length, 4 periods
    x = np.linspace(0, x_length, points)  
    zsum=np.zeros(points)
    yzero=np.zeros(points)
    sub_loc = [0.4,0.6,0.8,1]
    

    # Plot subcarriers
    for i in range(1,5):
        z_i = amp_list[i-1] * np.cos(i * (x-x_length/points*ani_step) + phase_list[i-1]*np.pi) # phase_list is minus number
        ax.plot(x, z_i, zs=0.2*(
            i+1), zdir='y', linewidth=1.0, alpha= 0.8, color=tableau_blues[i], label=r"$Sub_{{{index}}}: A_{{{amp}}}^I\cos({omega}t+\phi_{{{phi}}})$".format(index=i, amp=i, omega=i, phi=i))
        ax.plot(x, yzero, zs=0.2*(
            i+1), zdir='y', linewidth=0.5, alpha= 0.4, linestyle=':', color=tableau_blues[i])

        zsum += z_i
        
    # Root-Mean-Sqare and Peak-to-Average-Power-Ratio of the signal
    zrms=np.sqrt(np.mean(np.square(zsum)))
    #zrms=np.sqrt(zsum.dot(zsum)/zsum.size)
    papr=np.max(zsum)/zrms
    
    
    if t_check:
        ax.plot(x, zsum, zs=0, zdir='y', color='midnightblue', linewidth=2.0, label='Sum')
        ax.plot(x, yzero, zs=0, zdir='y', color='midnightblue', linestyle=':', linewidth=1.0, alpha=0.2)
        ax.plot(x, zrms*np.ones(points), zs=0, zdir='y', color='midnightblue', linestyle='--', linewidth=1.0, alpha=0.8, label='RMS level')
        ax.text(-2, 0, zrms-0.5, 'PAPR: %.1f' % papr , color='midnightblue', alpha=0.8, ha='center', va= 'bottom',fontsize=12)
    
    
    
    if f_check:
        ax.bar( sub_loc, amp_list, zs=12.6, zdir='x', color='darkviolet', width=0.01, label='Amplitude')
        
        # phase extension in -z direction
        #ax.bar( sub_loc, phase_list, zs=12.6, zdir='x', color='magenta', width=0.01, label='Phase') 
        
        # phase extension in +x direction, better 
        for i in range(len(phase_list)):
            phase_extensionx=np.linspace(0, - phase_list[i], 10)+x_length
            phase_extensiony=np.ones(10)*sub_loc[i]
            if i==1: ax.plot(phase_extensionx, phase_extensiony, zs=0, zdir='z', linewidth=1.5, alpha= 0.9,  color="magenta", label='Phase')
            else: ax.plot(phase_extensionx, phase_extensiony, zs=0, zdir='z', linewidth=1.5, alpha= 0.9,  color="magenta")
        



        for i in range(1,5): 
            #ax.text(12.6, a, b+0.05, r"(%d, %.1f $\pi$)" % (b,-c) , ha='center', va= 'bottom',fontsize=12)
            ax.text(12.6, sub_loc[i-1], amp_list[i-1]+0.05, 
                    r"$A_{{{index}}}^I$: %d".format(index=i) % amp_list[i-1] , 
                    ha='center', color='darkviolet', va= 'bottom',fontsize=12)
            ax.text(13.6-phase_list[i-1], sub_loc[i-1], -0.5, 
                    r"$\phi_{{{index}}}^I: %.1f \pi$)".format(index=i) % -phase_list[i-1] , 
                    ha='center', color='magenta', va= 'bottom',fontsize=12)


    # Adjust the appearance of the cube
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.1))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.1))
    ax.zaxis.set_pane_color((0.75, 0.75, 0.75, 0.15))
    ax.xaxis._axinfo["grid"]['linestyle'] = ":"
    ax.yaxis._axinfo["grid"]['linestyle'] = ":"
    ax.zaxis._axinfo["grid"]['linestyle'] = ":"




    # Make legend, set axes limits and labels
    ax.legend(bbox_to_anchor=(1.05, 1), loc=1, borderaxespad=0.)
    ax.set_xlim([0, 12.6])
    ax.set_ylim([0, 1])
    ax.set_zlim([0, 4])
    if t_check:
        ax.set_xlabel('\n' + r"$\mathscr{T}$ime", horizontalalignment='left', linespacing=3, fontsize=20)
    #ax.xaxis.set_label_coords(-0.5, 0.5)   # not functioning?
    if f_check: 
        ax.set_ylabel('\n' +'\t'+r"$\mathscr{F}$requency", horizontalalignment='right', linespacing=3, fontsize=20)
    ax.set_zlabel('Amplitude')
    
    ax.set_xticks([0, np.pi/2, np.pi, 1.5*np.pi, 2*np.pi, 
                2.5*np.pi, 3*np.pi, 3.5*np.pi, 4*np.pi])
    ax.set_xticklabels(
               [r'$0$', r'$+\dfrac{1}{2}t$', r'$+t$', r'$+\dfrac{3}{2}t$', 
                r'$+2t$', r'$+\dfrac{5}{2}t$', r'$+3t$', r'$+\dfrac{7}{2}t$', r'$+4t$'])
    ax.tick_params(axis='x', colors='grey')
    ax.set_yticks(sub_loc)
    ax.set_yticklabels([r'$Sub_{1}$',r'$Sub_{2}$',r'$Sub_{3}$', r'$Sub_{4}$'])
    ax.tick_params(axis='y', colors='grey')
    ax.set_zticks([-2,-1, 0, 1, 2, 4])
    ax.tick_params(axis='z', colors='grey')

    # Customize the view angle so it's easier to see that the scatter points lie
    # on the plane y=0
    ax.view_init(elev, azim)
    
    if save_check:
        fig.savefig('{index}.png'.format(index=ani_step))

    
    plt.show()
    
    
    
    
    
amp_max=3
phase_div=4
amp_w_list={}
phase_w_list={}
for i in range(1,5):
    amp_w_list["amp_w_{0}".format(i)] = widgets.IntSlider(min=0,max=amp_max,step=1,value=1,description=r"$A_{{{index}}}$".format(index=i),disabled=False,continuous_update=False)
    phase_w_list["phase_w_{0}".format(i)] = widgets.FloatSlider(min=0,max=2-1/phase_div ,step=2/phase_div,value=0,description=r"$\phi_{{{index}}}$".format(index=i),disabled=False, continuous_update=False)



elev_w = widgets.IntSlider(min=-90,max=180,step=5,value=40,description="Elev(view)",disabled=False,continuous_update=False)
azim_w = widgets.IntSlider(min=-180,max=180,step=5,value=-35,description="Azim(view)",disabled=False,continuous_update=False)

# ui6_layout = widgets.Layout(display='flex',
#                     flex_flow='row',
#                     align_items='stretch',
#                     width='70%')
#t_check = widgets.Checkbox(value=True, description=r"$\mathscr{T}$", layout=widgets.Layout(flex='1 1 auto', width='auto'), disabled=False)
#f_check = widgets.Checkbox(value=True, description=r"$\mathscr{F}$", layout=widgets.Layout(flex='1 1 auto', width='auto'), disabled=False)
t_check_w = widgets.Checkbox(value=True, description=r"$\mathscr{T}$", disabled=False)
f_check_w = widgets.Checkbox(value=True, description=r"$\mathscr{F}$", disabled=False)


# animation setup, ani_step is controlled by play_ani, which controls the movement of the subcarriers in ofdm3d
ani_step_w = widgets.IntSlider(min=0,max=256) 
#play_ani = widgets.Play(interval=300, value=0, min=0, max=256, step=8, layout=widgets.Layout(flex='8 1 auto', width='auto'), description="Press play", disabled=False)
play_ani_w = widgets.Play(interval=1500, value=0, min=0, max=256, step=8, description="Press play", disabled=False)
widgets.jslink((play_ani_w, 'value'), (ani_step_w, 'value'))


ui1 = widgets.HBox([amp_w_list['amp_w_1'], phase_w_list['phase_w_1']])
ui2 = widgets.HBox([amp_w_list['amp_w_2'], phase_w_list['phase_w_2']])
ui3 = widgets.HBox([amp_w_list['amp_w_3'], phase_w_list['phase_w_3']])
ui4 = widgets.HBox([amp_w_list['amp_w_4'], phase_w_list['phase_w_4']])
ui5 = widgets.HBox([elev_w, azim_w])
# ui6 = widgets.HBox([t_check, f_check, play_ani], layout=ui6_layout)  # for layout adjustment, not so useful
ui6 = widgets.HBox([t_check_w, f_check_w, play_ani_w])




fun_out = widgets.interactive_output(ofdm3d, {'amp_1': amp_w_list['amp_w_1'], 'amp_2': amp_w_list['amp_w_2'], 'amp_3': amp_w_list['amp_w_3'], 'amp_4': amp_w_list['amp_w_4'], 
                                              'phase_1': phase_w_list['phase_w_1'], 'phase_2': phase_w_list['phase_w_2'], 'phase_3': phase_w_list['phase_w_3'], 'phase_4': phase_w_list['phase_w_4'],
                                              'elev': elev_w, 'azim': azim_w, 't_check': t_check_w, 'f_check': f_check_w, 
                                              'ani_step': ani_step_w})



if True: display(ui1, ui2, ui3, ui4, ui5, ui6, fun_out)



Output()

In [2]:
# Visualization of OFDM with its subcarriers
# from sinusoidal to complex waves
# finalized version @20180417
# by HP

#from __future__ import print_function
#from ipywidgets import interact, interactive, fixed, interact_manual
%matplotlib inline
import ipywidgets as widgets
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt



### Coloring
# Color from http://www.randalolson.com/2014/06/28/how-to-make-beautiful-data-visualizations-in-python-with-matplotlib/
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
         (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
         (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
         (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
         (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)] 
# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.)  
# from http://www.color-hex.com/color-palette/1294
tableau_blues = [(1,31,75), (3,57,108), (0,91,150), (100,151,177), (179,205,224)]
for i in range(len(tableau_blues)):    
    r, g, b = tableau_blues[i]    
    tableau_blues[i] = (r / 255., g / 255., b / 255.)
    
# from http://www.colourlovers.com/palette/73804/sisters_crayons
tableau_greens = [(72,121,87), (93,148,106), (108,172,123), (125,197,142), (143,223,161)]
for i in range(len(tableau_greens)):    
    r, g, b = tableau_greens[i]    
    tableau_greens[i] = (r / 255., g / 255., b / 255.)


def ofdm3d(amp_1=1, phase_1=0, 
           amp_2=1, phase_2=0, 
           amp_3=1, phase_3=0, 
           amp_4=1, phase_4=0,
           amp_s_1=1, amp_s_2=1,amp_s_3=1, amp_s_4=1,
           elev=40, azim=-35,
           t_check=True, f_check=True, sin_check=1, complex_check=1,
           ani_step=0, save_check=False
          ):
    
    # To remove part of the margin of the plot-out cube area
    # source: https://stackoverflow.com/questions/16488182/removing-axes-margins-in-3d-plot
    ###patch start###   
    from mpl_toolkits.mplot3d.axis3d import Axis
    if not hasattr(Axis, "_get_coord_info_old"):
        def _get_coord_info_new(self, renderer):
            mins, maxs, centers, deltas, tc, highs = self._get_coord_info_old(renderer)
            mins += deltas / 4
            maxs -= deltas / 4
            return mins, maxs, centers, deltas, tc, highs
        Axis._get_coord_info_old = Axis._get_coord_info  
        Axis._get_coord_info = _get_coord_info_new
    ###patch end###
    
    # This is for solving the latex rendering problem. not very successful
    # https://stackoverflow.com/questions/14324477/bold-font-weight-for-latex-axes-label-in-matplotlib/16019418
    import matplotlib as mpl
    mpl.rc('text', usetex=False)
    #mpl.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]    # latex error, maybe has to do with installation
    
    
    # Config of the figure
    fig = plt.figure(figsize=(12,6), dpi=150)
    ax = fig.gca(projection='3d')



    # Parameters
    amp_list=[amp_1,amp_2,amp_3,amp_4]
    amp_list_s=[amp_s_1,amp_s_2,amp_s_3,amp_s_4]
    phase_list=[-phase_1,-phase_2,-phase_3,-phase_4]
    points=512
    x_length=12.56    # 12.56 for total length, 4 periods
    x = np.linspace(0, x_length, points)  
    zsum=np.zeros(points)
    zsum_sin=np.zeros(points)
    zsum_com=np.zeros(points)
    yzero=np.zeros(points)
    sub_loc = [0.4,0.6,0.8,1]
    

    # Plot subcarriers
    for i in range(1,5):
        z_i = amp_list[i-1] * np.cos(i * (x-x_length/points*ani_step) + phase_list[i-1]*np.pi) # phase_list is minus number
        z_sin_i = amp_list_s[i-1] * np.sin(i * (x-x_length/points*ani_step) + phase_list[i-1]*np.pi)
        ax.plot(x, yzero, zs=0.2*(
            i+1), zdir='y', linewidth=0.5, alpha= 0.4, linestyle=':', color=tableau_blues[i])       
        if complex_check:
            z_com_i=z_i+z_sin_i
            ax.plot(x, z_com_i, zs=0.2*(i+1), zdir='y', linewidth=1.0, alpha= 0.8, color=tableau_blues[i], 
                    label=r"$Sub_{{{index}}}: \left | A_{{{amp}}} \right |exp(\angle A_{{{omega}}}t)$".format(index=i, amp=i, omega=i))
        else:
                
            ax.plot(x, z_i, zs=0.2*(
                i+1), zdir='y', linewidth=1.0, alpha= 0.8, color=tableau_blues[i], label=r"$Sub_{{{index}}}: A_{{{amp}}}^I\cos({omega}t)$".format(index=i, amp=i, omega=i))

            if sin_check: 

                ax.plot(x, z_sin_i, zs=0.2*(i+1), zdir='y', linewidth=1.0, alpha= 0.8, color=tableau_greens[i], 
                        label=r"$Sub_{{{index}}}: A_{{{amp}}}^Q\sin({omega}t)$".format(index=i, amp=i, omega=i))
          

        zsum += z_i
        if sin_check: zsum_sin += z_sin_i
    if complex_check:
        zsum_com=zsum+zsum_sin
        
    
    # Root-Mean-Sqare and Peak-to-Average-Power-Ratio of the signal
    zrms=np.sqrt(np.mean(np.square(zsum)))
    #if sin_check: zrms=zrms+np.sqrt(np.mean(np.square(zsum_sin)))
    if complex_check:
        zsum_com=zsum+zsum_sin
        zrms=np.sqrt(zsum.dot(zsum_com)/zsum_com.size)
        papr=np.max(zsum_com)/zrms
    else:
        zrms=np.sqrt(zsum.dot(zsum)/zsum.size)
        papr=np.max(zsum)/zrms
    
    
    if t_check:
        if complex_check:
            
            ax.plot(x, zsum_com, zs=0, zdir='y', color='darkblue', linewidth=2.0, label='Sum of Complex Waves')
        else:
            if sin_check:
                ax.plot(x, zsum_sin, zs=0, zdir='y', color='royalblue', linewidth=2.0, label='Sum of Sin Waves')
            ax.plot(x, zsum, zs=0, zdir='y', color='midnightblue', linewidth=2.0, label='Sum of Cos Waves')
            

        ax.plot(x, yzero, zs=0, zdir='y', color='midnightblue', linestyle=':', linewidth=1.0, alpha=0.2)
        
        ax.plot(x, zrms*np.ones(points), zs=0, zdir='y', color='midnightblue', linestyle='--', linewidth=1.0, alpha=0.8, label='RMS level')
        ax.text(-2, 0, zrms-0.5, 'PAPR: %.1f' % papr , color='midnightblue', alpha=0.8, ha='center', va= 'bottom',fontsize=12)
    
    
    
    if f_check:
        
        if sin_check:
#             sub_loc_s=[0.35,0.55,0.75,0.95]
            sub_loc = [0.4,0.6,0.8,1]
#             for i in range(1, (len(sub_loc)-1)):
#                 sub_loc_s[i] = sub_loc[i]-0.05
#                 sub_loc[i] = sub_loc[i]+0.05
            ax.bar( sub_loc, amp_list_s, zs=0, zdir='x', color='darkgoldenrod', width=0.01, label='Amplitude Q')
        ax.bar( sub_loc, amp_list, zs=12.6, zdir='x', color='darkviolet', width=0.01, label='Amplitude I')
        
        # phase extension in -z direction
        #ax.bar( sub_loc, phase_list, zs=12.6, zdir='x', color='magenta', width=0.01, label='Phase') 
        
        # phase extension in +x direction, better 
        for i in range(len(phase_list)):
            phase_extensionx=np.linspace(0, - phase_list[i], 10)+x_length
            phase_extensiony=np.ones(10)*sub_loc[i]
            if i==1: ax.plot(phase_extensionx, phase_extensiony, zs=0, zdir='z', linewidth=1.5, alpha= 0.9,  color="magenta", label='Phase')
            else: ax.plot(phase_extensionx, phase_extensiony, zs=0, zdir='z', linewidth=1.5, alpha= 0.9,  color="magenta")
        



        for i in range(1,5): 
            #ax.text(12.6, a, b+0.05, r"(%d, %.1f $\pi$)" % (b,-c) , ha='center', va= 'bottom',fontsize=12)
            ax.text(12.6, sub_loc[i-1], amp_list[i-1]+0.05, 
                    r"$A_{{{index}}}^I$: %d".format(index=i) % amp_list[i-1] , 
                    ha='center', color='darkviolet', va= 'bottom',fontsize=12)
            if sin_check:
                ax.text(0, sub_loc[i-1], amp_list_s[i-1]+0.05, 
                    r"$A_{{{index}}}^Q$: %d".format(index=i) % amp_list_s[i-1] , 
                    ha='center', color='darkgoldenrod', va= 'bottom',fontsize=12)
#             ax.text(13.6-phase_list[i-1], sub_loc[i-1], -0.5, 
#                     r"$\phi_{{{index}}}: %.1f \pi$)".format(index=i) % -phase_list[i-1] , 
#                     ha='center', color='magenta', va= 'bottom',fontsize=12)


    # Adjust the appearance of the cube
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.1))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.1))
    ax.zaxis.set_pane_color((0.75, 0.75, 0.75, 0.15))
    ax.xaxis._axinfo["grid"]['linestyle'] = ":"
    ax.yaxis._axinfo["grid"]['linestyle'] = ":"
    ax.zaxis._axinfo["grid"]['linestyle'] = ":"




    # Make legend, set axes limits and labels
    ax.legend(bbox_to_anchor=(1.05, 1), loc=1, borderaxespad=0.)
    ax.set_xlim([0, 12.6])
    ax.set_ylim([0, 1])
    ax.set_zlim([0, 4])
    if t_check:
        ax.set_xlabel('\n' + r"$\mathscr{T}$ime", horizontalalignment='left', linespacing=3, fontsize=20)
    #ax.xaxis.set_label_coords(-0.5, 0.5)   # not functioning?
    if f_check: 
        ax.set_ylabel('\n' +'\t'+r"$\mathscr{F}$requency", horizontalalignment='right', linespacing=3, fontsize=20)
    ax.set_zlabel('Amplitude')
    
    ax.set_xticks([0, np.pi/2, np.pi, 1.5*np.pi, 2*np.pi, 
                2.5*np.pi, 3*np.pi, 3.5*np.pi, 4*np.pi])
    ax.set_xticklabels(
               [r'$0$', r'$+\dfrac{1}{2}t$', r'$+t$', r'$+\dfrac{3}{2}t$', 
                r'$+2t$', r'$+\dfrac{5}{2}t$', r'$+3t$', r'$+\dfrac{7}{2}t$', r'$+4t$'])
    ax.tick_params(axis='x', colors='grey')
    ax.set_yticks(sub_loc)
    ax.set_yticklabels([r'$Sub_{1}$',r'$Sub_{2}$',r'$Sub_{3}$', r'$Sub_{4}$'])
    ax.tick_params(axis='y', colors='grey')
    ax.set_zticks([-2,-1, 0, 1, 2, 4])
    ax.tick_params(axis='z', colors='grey')

    # Customize the view angle so it's easier to see that the scatter points lie
    # on the plane y=0
    ax.view_init(elev, azim)
    
    if save_check:
        fig.savefig('{index}.png'.format(index=ani_step))

    
    plt.show()
    
    
    
    
    
amp_max=3
phase_div=4
amp_w_list={}
amp_s_w_list={}
phase_w_list={}
for i in range(1,5):
    amp_w_list["amp_w_{0}".format(i)] = widgets.IntSlider(min=0,max=amp_max,step=1,value=1,description=r"$A_{{{index}}}^I$".format(index=i),disabled=False,continuous_update=False)
    amp_s_w_list["amp_s_w_{0}".format(i)] = widgets.IntSlider(min=0,max=amp_max,step=1,value=1,description=r"$A_{{{index}}}^Q$".format(index=i),disabled=False,continuous_update=False)    
    phase_w_list["phase_w_{0}".format(i)] = widgets.FloatSlider(min=0,max=2-1/phase_div ,step=2/phase_div,value=0,description=r"$\phi_{{{index}}}$".format(index=i),disabled=False, continuous_update=False)



elev_w = widgets.IntSlider(min=-90,max=180,step=5,value=40,description="Elev(view)",disabled=False,continuous_update=False)
azim_w = widgets.IntSlider(min=-180,max=180,step=5,value=-35,description="Azim(view)",disabled=False,continuous_update=False)

# ui6_layout = widgets.Layout(display='flex',
#                     flex_flow='row',
#                     align_items='stretch',
#                     width='70%')
#t_check = widgets.Checkbox(value=True, description=r"$\mathscr{T}$", layout=widgets.Layout(flex='1 1 auto', width='auto'), disabled=False)
#f_check = widgets.Checkbox(value=True, description=r"$\mathscr{F}$", layout=widgets.Layout(flex='1 1 auto', width='auto'), disabled=False)
t_check_w = widgets.Checkbox(value=True, description=r"$\mathscr{T}$", disabled=False)
f_check_w = widgets.Checkbox(value=True, description=r"$\mathscr{F}$", disabled=False)
sin_check_w = widgets.Checkbox(value=True, description=r"Sin", disabled=False)
complex_check_w = widgets.Checkbox(value=True, description=r"Complex", disabled=False)



# animation setup, ani_step is controlled by play_ani, which controls the movement of the subcarriers in ofdm3d
ani_step_w = widgets.IntSlider(min=0,max=256) 
#play_ani = widgets.Play(interval=300, value=0, min=0, max=256, step=8, layout=widgets.Layout(flex='8 1 auto', width='auto'), description="Press play", disabled=False)
play_ani_w = widgets.Play(interval=1500, value=0, min=0, max=256, step=8, description="Press play", disabled=False)
widgets.jslink((play_ani_w, 'value'), (ani_step_w, 'value'))


ui1 = widgets.HBox([amp_w_list['amp_w_1'], amp_s_w_list['amp_s_w_1'],]) #phase_w_list['phase_w_1']])
ui2 = widgets.HBox([amp_w_list['amp_w_2'], amp_s_w_list['amp_s_w_2'],]) #phase_w_list['phase_w_2']])
ui3 = widgets.HBox([amp_w_list['amp_w_3'], amp_s_w_list['amp_s_w_3'],]) #phase_w_list['phase_w_3']])
ui4 = widgets.HBox([amp_w_list['amp_w_4'], amp_s_w_list['amp_s_w_4'],]) #phase_w_list['phase_w_4']])
ui5 = widgets.HBox([elev_w, azim_w])
# ui6 = widgets.HBox([t_check, f_check, play_ani], layout=ui6_layout)  # for layout adjustment, not so useful
ui6 = widgets.HBox([t_check_w, f_check_w, sin_check_w, complex_check_w, play_ani_w])




fun_out = widgets.interactive_output(ofdm3d, {'amp_1': amp_w_list['amp_w_1'], 'amp_2': amp_w_list['amp_w_2'], 'amp_3': amp_w_list['amp_w_3'], 'amp_4': amp_w_list['amp_w_4'], 
                                              'amp_s_1': amp_s_w_list['amp_s_w_1'],'amp_s_2': amp_s_w_list['amp_s_w_2'],'amp_s_3': amp_s_w_list['amp_s_w_3'],'amp_s_4': amp_s_w_list['amp_s_w_4'],
                                              #'phase_1': phase_w_list['phase_w_1'], 'phase_2': phase_w_list['phase_w_2'], 'phase_3': phase_w_list['phase_w_3'], 'phase_4': phase_w_list['phase_w_4'],
                                              'elev': elev_w, 'azim': azim_w, 't_check': t_check_w, 'f_check': f_check_w, 'sin_check': sin_check_w, 'complex_check': complex_check_w, 
                                              'ani_step': ani_step_w})



if True: display(ui1, ui2, ui3, ui4, ui5, ui6, fun_out)



Output()